In [2]:
 %%capture
 !pip install langchain==0.2.5 faiss-gpu==1.7.2 cohere==5.5.8 langchain-community==0.2.5 rank_bm25==0.2.2 sentence-transformers==3.0.1
 !CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.78

In [3]:
import cohere

api_key = ""

co = cohere.Client(api_key)

In [14]:
text = """
As of the starting of our computations, there were 218 zeolite
framework types catalogued by IZA. By excluding 9 interrupted
framework topologies and RWY (an unusual topology existing only as
chalcogenides) but including BEB (a polymorph B of zeolite beta),
209 topologies were selected for our computations. 32 Coordinates of
each topology (in pure silica compositions) were obtained as CIF ﬁles
downloaded from IZA’s database. 4 The energy minimizations were
performed on the GULP program 33 with the SLC potential under the
constant pressure condition. The framework energies reported here
were calculated relative to α-quartz. The computational cells were
converted to a P1 space group symmetry to allow the atomic positions
and other crystallographic parameters to be altered during the
molecular mechanics optimization. For pure silica compositions, the
computational cells were in a unit-cell size, while the super cells were
used for aluminosilicate counterparts.
The super cells were created by expanding each unit cell until the
number of T atoms reached the following conditions: (i) to increase
the randomness of Al distribution, the number of T atoms was more
than 100; and (ii) to achieve the targeted substituting contents of Al,
the number of T atoms was the multiples of (Si + Al)/Al molar ratios.
Because the required computational resources are increased as the
number of T atoms in the super cells increases, we tried to avoid
unnecessary expansion of the unit cells by choosing the targeted
substituting contents of Al with the (Si + Al)/Al molar ratios being the
power of 2 (i.e., 2, 4, 8, 16, and 32), resulting in the Si/Al of 1, 3, 7, 15,
and 31, respectively. In extreme cases in which the number of T atoms
in the unit cells is odd numbers (i.e., OSO and VET topologies), the
super cells were 32 times larger than their unit cells for achieving the
Si/Al of 31
"""

texts = text.split('.')

texts = [t.strip(' \n') for t in texts]

In [15]:
import numpy as np

# Get the embeddings
response = co.embed(
  texts=texts,
  input_type="search_document",
).embeddings

embeds = np.array(response)
print(embeds.shape)

(14, 4096)


In [16]:
import faiss

dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.float32(embeds))

In [17]:
import pandas as pd

def search(query, number_of_results=1):

  query_embed = co.embed(texts=[query],
                input_type="search_query",).embeddings[0]
  distances , similar_item_ids = index.search(np.float32([query_embed]), number_of_results)

  texts_np = np.array(texts)
  results = pd.DataFrame(data={'texts': texts_np[similar_item_ids[0]],
                              'distance': distances[0]})

  print(f"Query:'{query}'\nNearest neighbors:")
  return results

In [18]:
query = "How was the super cells created?"
results = search(query)
results

Query:'How was the super cells created?'
Nearest neighbors:


,texts,distance
0,The super cells were created by expanding each...,6072.244629
